In [2]:
#Importing all the required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
from datetime import datetime
import pandas_datareader as pdr

C:\Users\Hp\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
yf.pdr_override()
stock = input("Enter a stock ticker symbol: ")
print(stock)

Enter a stock ticker symbol: 



In [4]:
start_date = input("Give me a start date(YYYY-MM-DD):")
print(start_date)
end_date = input("Give me an end date(YYYY-MM-DD):")
print(end_date)

KeyboardInterrupt: Interrupted by user

In [27]:
# df stands for data frame
df = yf.download(stock, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [31]:
# Moving Average
ma = 50
smaString = "Sma_"+str(ma)
df[smaString] = df.iloc[:,4].rolling(window = ma).mean()
# We will be cutting out the first 50 rows as it won't show any average
df = df.iloc[ma:]
print(df)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2019-05-28   44.730000   45.147499   44.477501   44.557499   43.115597   
2019-05-29   44.105000   44.837502   44.000000   44.345001   42.909969   
2019-05-30   44.487499   44.807499   44.167500   44.575001   43.132530   
2019-05-31   44.057499   44.497501   43.747501   43.767502   42.351158   
2019-06-03   43.900002   44.480000   42.567501   43.325001   41.922985   
...                ...         ...         ...         ...         ...   
2022-12-23  130.919998  132.419998  129.639999  131.860001  131.127060   
2022-12-27  131.380005  131.410004  128.720001  130.029999  129.307236   
2022-12-28  129.669998  131.029999  125.870003  126.040001  125.339409   
2022-12-29  127.989998  130.479996  127.730003  129.610001  128.889572   
2022-12-30  128.410004  129.949997  127.430000  129.929993  129.207794   

               Volume      Sma_50  
D

<ipython-input-31-6c41ed141d13>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[smaString] = df.iloc[:,4].rolling(window = ma).mean()


In [39]:
numH = 0
numC = 0
for i in df.index:
    #print(df.iloc[:,4][i])
    #print(df[smaString][i])
    if (df["Adj Close"][i] > df[smaString][i]):
        print("The asjusted close is higher")
        numH += 1
    else:
        print("The adjusted close is lower")
        numC += 1
print(str(numH))
print(str(numC))

The adjusted close is lower
The adjusted close is lower
The adjusted close is lower
The adjusted close is lower
The adjusted close is lower
The adjusted close is lower
The adjusted close is lower
The adjusted close is lower
The adjusted close is lower
The adjusted close is lower
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The adjusted close is lower
The adjusted close is lower
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is higher
The asjusted close is high

In [40]:
# Back-testing strategy
emasUsed = [3,5,8,10,12,15,30,35,40,45,50,60]
for x in emasUsed:
    ema = x
    df["Ema"+str(ema)] = round(df.iloc[:,4].ewm(span = ema, adjust = False).mean(),2)
print(df.tail())

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-12-23  130.919998  132.419998  129.639999  131.860001  131.127060   
2022-12-27  131.380005  131.410004  128.720001  130.029999  129.307236   
2022-12-28  129.669998  131.029999  125.870003  126.040001  125.339409   
2022-12-29  127.989998  130.479996  127.730003  129.610001  128.889572   
2022-12-30  128.410004  129.949997  127.430000  129.929993  129.207794   

              Volume      Sma_50    Ema3    Ema5    Ema8   Ema10   Ema12  \
Date                                                                       
2022-12-23  63814900  143.258228  131.88  132.76  134.25  135.18  136.01   
2022-12-27  69007800  143.096715  130.59  131.61  133.15  134.11  134.98   
2022-12-28  85438400  142.775825  127.97  129.52  131.42  132.52  133.49   
2022-12-29  75703700  142.499332  128.43  129.31  130.86  131.86  132.78   
2022-12-30  77034200  142

In [45]:
pos = 0
num = 0
percentchange = []

for i in df.index:
    cmin = min(df["Ema3"][i],df["Ema5"][i],df["Ema8"][i],df["Ema10"][i],df["Ema12"][i],df["Ema15"][i])
    cmax = max(df["Ema30"][i],df["Ema35"][i],df["Ema40"][i],df["Ema45"][i],df["Ema50"][i],df["Ema60"][i])
    
    close = df["Adj Close"][i]
    
    if (cmin > cmax):
        print("Red White Blue")
        if (pos == 0):
            bp = close
            pos = 1
            print("Buying now at "+str(bp))
    elif (cmin < cmax):
        print("Blue White Red")
        if (pos == 1):
            pos = 0
            sp = close
            print("Selling now at "+str(sp))
            pc = (sp/bp-1)*100
            percentchange.append(pc)
    if (num == df["Adj Close"].count() - 1 and pos == 1):
        sp = close
        print("Selling now at "+str(sp))
        pc = (sp/bp-1)*100
        percentchange.append(pc)
        
    num += 1
print(percentchange)

Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Red White Blue
Buying now at 44.806541442871094
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Blue White Red
Selling now at 46.77084732055664
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Blue White Red
Red White Blue
Buying now at 50.74396514892578
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Red White Blue
Re

In [53]:
gains = []
ng = 0
losses = 0
nl = 0
totalR = 1

for i in percentchange:
    if (i > 0):
        gains += i
        ng += 1
    else:
        losses += i
        nl += 1
    totalR = totalR*((i/100)+1)
totalR = round((totalR-1)*100,2)

#For gains
if (ng > 0):
    avgGain = gains/ng
    maxR = str(max(percentchange))
else:
    avgGain = 0
    maxR = "undefined"

#For losses
if (ng > 0):
    avgLoss = losses/nl
    maxL = str(min(percentchange))
    ratio = str(-avgGain/avgLoss)
else:
    avgLoss = 0
    maxL = "undefined"
    ratio = "inf"

if (ng > 0 or nl > 0):
    battingAvg = ng/(ng+nl)
else:
    battingAvg = 0

print()
print("Results for "+ stock +" going back to "+str(df.index[0])+", Sample size: "+str(ng+nl)+" trades")
print("EMAs used: "+str(emasUsed))
print("Batting Avg: "+ str(battingAvg))
print("Gain/loss ratio: "+ ratio)
print("Average Gain: "+ str(avgGain))
print("Average Loss: "+ str(avgLoss))
print("Max Return: "+ maxR)
print("Max Loss: "+ maxL)
print("Total return over "+str(ng+nl)+ " trades: "+ str(totalR)+"%" )
#print("Example return Simulating "+str(n)+ " trades: "+ str(nReturn)+"%" )
print()


Results for AAPL going back to 2019-05-28 00:00:00, Sample size: 13 trades
EMAs used: [3, 5, 8, 10, 12, 15, 30, 35, 40, 45, 50, 60]
Batting Avg: 0.5384615384615384
Gain/loss ratio: []
Average Gain: []
Average Loss: -2.592855947793025
Max Return: 58.48523550135345
Max Loss: -5.133932076528991
Total return over 13 trades: 200.22%

